# 

## 1. Make sure you have the following packages in your environment installed

In [1]:
# !pip install --upgrade pip
# !pip install langchain==0.3.25 langchain-community==0.3.25 langchain-ollama==0.3.3 chromadb==1.0.12

In [2]:
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

## 2. Load your documents

In [3]:
loader = DirectoryLoader(
    "./documents",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)
docs = loader.load()
print(f"One single document:\n\n'{docs[1].page_content}'\n\nFrom the text file:\n'{docs[1].metadata['source']}'")

One single document:

'CLEVER GRETEL

There was once a cook named Gretel, who wore shoes with red heels, and when she walked out with them on, she turned herself this way and that, was quite happy and thought: ‘You certainly are a pretty girl!’ And when she came home she drank, in her gladness of heart, a draught of wine, and as wine excites a desire to eat, she tasted the best of whatever she was cooking until she was satisfied, and said: ‘The cook must know what the food is like.’

It came to pass that the master one day said to her: ‘Gretel, there is a guest coming this evening; prepare me two fowls very daintily.’ ‘I will see to it, master,’ answered Gretel. She killed two fowls, scalded them, plucked them, put them on the spit, and towards evening set them before the fire, that they might roast. The fowls began to turn brown, and were nearly ready, but the guest had not yet arrived. Then Gretel called out to her master: ‘If the guest does not come, I must take the fowls away from 

## 3. Chunk your documents

In [4]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
chunks = splitter.split_documents(docs)
print(f"One single document chunk:\n'{chunks[5].page_content}'\n\nFrom the file:\n'{chunks[5].metadata['source']}'")

One single document chunk:
'CLEVER GRETEL

There was once a cook named Gretel, who wore shoes with red heels, and when she walked out with them on, she turned herself this way and that, was quite happy and thought: ‘You certainly are a pretty girl!’ And when she came home she drank, in her gladness of heart, a draught of wine, and as wine excites a desire to eat, she tasted the best of whatever she was cooking until she was satisfied, and said: ‘The cook must know what the food is like.’'

From the file:
'documents/clever_gretel.txt'


## 4. Embed and index chunks in a vector database
Make sure to execute `ollama pull all-minilm` in the terminal before running the following snippet.

In [5]:
# !ollama pull all-minilm
# !ollama pull nomic-embed-text
# !ollama pull mxbai-embed-large

In [6]:
# model = "all-minilm"         # 23M parameters
# model = "nomic-embed-text"   # 137M parameters
model = "mxbai-embed-large"   # 334M parameters

# Create a vector store to store the embeddings or our chunked documents
embed_model = OllamaEmbeddings(model=model, base_url="http://127.0.0.1:11434")
vector_store = Chroma.from_documents(chunks, embed_model)
collection = vector_store._collection

print("Total amount of chunks / embeddings:", collection.count())

Total amount of chunks / embeddings: 63


In [7]:
all_data = collection.get(include=["documents", "embeddings"])
print(f"One single document:\n{all_data['documents'][1]}\n")
print(f"The document's embedding:\n{all_data['embeddings'][1]}\n")


One single document:
It happened that the cat met the fox in a forest, and as she thought to herself: ‘He is clever and full of experience, and much esteemed in the world,’ she spoke to him in a friendly way. ‘Good day, dear Mr Fox, how are you? How is all with you? How are you getting on in these hard times?’ The fox, full of all kinds of arrogance, looked at the cat from head to foot, and for a long time did not know whether he would give any answer or not. At last he said: ‘Oh, you wretched beard-cleaner, you piebald fool,

The document's embedding:
[ 0.04361834  0.0033501   0.01679566 ...  0.01759302  0.00692107
 -0.00018101]



## 5. Create a Retrieval-QA chain
Make sure to execute `ollama pull gemma3:1b` in the terminal before running the following snippet.

In [8]:
# !ollama pull gemma3:1b
# !ollama pull llama3.2
# !ollama pull mistral

In [9]:
# model = "gemma3:1b"     # 1B parameters
# model = "llama3.2"    # 3B parameters
model = "mistral"     # 7B parameters

# Create a language model with an endpoint, and a RetrievalQA chain which takes the language model,
# and the vector store containing our documents.
llm = OllamaLLM(
    model=model,
    base_url="http://127.0.0.1:11434",
    temperature=0.9,
    top_p=0.95,
    top_k=46,
    # num_predict=256,
    )

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(search_kwargs={"k": 10}), # We set the number of chunks to return k
    chain_type="stuff",            # or "map_reduce", "refine", etc.
    return_source_documents=True,  # if you want the source chunks back
)
print(llm)

OllamaLLM
Params: {}


# 6 Ask questions!
Some examples:
### 6.1. "Who is Clever Hans in the Grimm Fairy Tales?"

In [ ]:
# Without RAG
query = "Who is Clever Hans in the Grimm Fairy Tales?"
result = llm.invoke(query)
print(f"Answer: {result}")

Answer:  There appears to be some confusion. Clever Hans was not a character in the Grimm fairy tales. Instead, Clever Hans was a horse that lived during the late 19th and early 20th centuries, who became famous for his ability to perform mathematical calculations by tapping his hoof when asked a question. The story of Clever Hans is more about psychology and human-animal communication rather than a Grimm fairy tale.


In [ ]:
# With RAG
result = qa_chain.invoke({"query": query})
print(f"Answer: {result['result']}\n")
print(f"Source: {result['source_documents']}")

Answer:  Clever Hans is one of the characters in the Grimm Fairy Tales. He is a horse with the ability to solve mathematical problems and other intellectual tasks. The context provided above comes from another tale featuring this character, showing his interactions with Gretel and his mother rather than his math skills.

Source: [Document(metadata={'source': 'documents/clever_hans.txt'}, page_content='CLEVER HANS\n\nThe mother of Hans said: ‘Whither away, Hans?’ Hans answered: ‘To Gretel.’ ‘Behave well, Hans.’ ‘Oh, I’ll behave well. Goodbye, mother.’ ‘Goodbye, Hans.’ Hans comes to Gretel. ‘Good day, Gretel.’ ‘Good day, Hans. What do you bring that is good?’ ‘I bring nothing, I want to have something given me.’ Gretel presents Hans with a needle, Hans says: ‘Goodbye, Gretel.’ ‘Goodbye, Hans.’'), Document(metadata={'source': 'documents/clever_hans.txt'}, page_content='‘Whither away, Hans?’ ‘To Gretel, mother.’ ‘Behave well, Hans.’ ‘Oh, I’ll behave well. Goodbye, mother.’ ‘Goodbye, Hans.’

### 6.2 "Cite what the frog says in the “The Frog Prince” in the Grimm Fairy Tales when he tries to visit the princess"

In [ ]:
# Without RAG
query = "Cite what the frog says in the “The Frog Prince” in the Grimm Fairy Tales when he tries to visit the princess"
result = llm.invoke(query)
print(f"Answer: {result}")

Answer:  In the Grimm Fairy Tale "The Frog Prince," when the transformed frog attempts to visit the Princess, the text goes as follows:

"Then the Frog climbed up on a stone and said to her: 'I am a nobleman turned into a Frog by a wicked enchanter's spell. A princess has to kiss me three times to make me a man again, and I shall reward you richly for it.'"

Here's the full link to the complete tale: https://www.pitt.edu/~dash/grimm-frogprince.html


In [ ]:
# With RAG
result = qa_chain.invoke({"query": query})
print(f"Answer: {result['result']}\n")
print(f"Source: {result['source_documents']}")
### 6.3. What is the fairy tale 'Fundevogel' about?

Answer:  The frog in "The Frog Prince" from the Grimm Fairy Tales says the following when he tries to visit the princess:

1. First time: "Open the door, my princess dear,
  Open the door to thy true love here!
  And mind the words that thou and I said
  By the fountain cool, in the greenwood shade."

2. Second time: (The frog repeats the same verse.)
"Open the door, my princess dear,
  Open the door to thy true love here!
  And mind the words that thou and I said
  By the fountain cool, in the greenwood shade."

Source: [Document(metadata={'source': 'documents/the_frog_prince.txt'}, page_content='THE FROG-PRINCE'), Document(metadata={'source': 'documents/the_frog_prince.txt'}, page_content='He told her that he had been enchanted by a spiteful fairy, who had changed him into a frog; and that he had been fated so to abide till some princess should take him out of the spring, and let him eat from her plate, and sleep upon her bed for three nights. ‘You,’ said the prince, ‘have broken his

### 6.3 "What is the fairy tale 'Fundevogel' about?"

In [ ]:
# Without RAG
query = "What is the fairy tale 'Fundevogel' about?"
result = llm.invoke(query)
print(f"Answer: {result}")

Answer:  "Fundevogel" is a German fairy tale that originates from the Brothers Grimm collection. The title translates to "Peacock Peter."

The story follows a peacock who loses one of his feathers and goes on a journey to find another one, as he wants to be beautiful like the other peacocks again. He meets various animals along the way and asks them if they know where he can find another feather. A cat tells him that there are plenty of peacock feathers in the castle of the king's daughter. The peacock pretends to be blind and convinces the king's daughter to show him her most beautiful feathers, ultimately using his sight to seduce her.

The peacock then convinces the princess to marry him, on condition that he can only touch her on their wedding day. They live together happily, and on the wedding day, the peacock grabs the princess's hand as they walk down the aisle and suddenly regains his sight. He immediately recognizes her for who she truly is: the cat who had given him the infor

In [ ]:
# With RAG
result = qa_chain.invoke({"query": query})
print(f"Answer: {result['result']}\n")
print(f"Source: {result['source_documents']}")

Answer:  The fairy tale 'Fundevogel' is a German folktale about a forester who finds and raises a child named Fundevogel, whom he found on a tree. Fundevogel later turns out to be a prince enchanted by a witch, and the story follows their journey together and how they eventually break the curse and live happily ever after. Along the way, Fundevogel and his friend Lina use their bond to transform into various objects to avoid being caught by the old cook who wants to harm them.

Source: [Document(metadata={'source': 'documents/fundevogel.txt'}, page_content='The forester climbed up, brought the child down, and thought to himself: ‘You will take him home with you, and bring him up with your Lina.’ He took it home, therefore, and the two children grew up together. And the one, which he had found on a tree was called Fundevogel, because a bird had carried it away. Fundevogel and Lina loved each other so dearly that when they did not see each other they were sad.'), Document(metadata={'sour